# C++ Introduction - Polymorphism

In [1]:
#include <string>
#include <iostream>
#include <sstream>
using namespace std;

### Inheritance
- Syntax
  - `class Child : public Base {}`
  - just use `public`: it means "leave things unchanged"
- `protected`
- Overriding methods
  - same name, return type and parameters
  - calling parent method from child
- Constructors and initialization lists

### Polymorphism
- pointers to base class
- `virtual`

In [2]:
class Book {
    private:
    string title;
    string author;
    int year;
    
    public:
    Book(string title, string author, int year) : title(title), author(author), year(year) {}
    Book(Book const& other) : title(other.title), author(other.author), year(other.year) { }
    Book & operator=(Book const& other) {
        this->title = other.title;
        this->author = other.author;
        this->year = other.year;
        return *this;
    }
    
    string getTitle() const { return this->title; }
    string getAuthor() const { return this->author; }
    int getYear() const { return this->year; }
    
    string str() const {
        stringstream ss;
        ss << this->title << " by " << this->author << " (" << this->year << ")";
        return ss.str();
    }
}

In [3]:
class GraphicNovel : public Book {
    private:
    string illustrator;
    public:
    GraphicNovel(string title, string author, int year, string illustrator) 
        : Book(title, author, year), illustrator(illustrator) {}
    
    string str() const {
        stringstream ss;
        ss << Book::str() << " (Illustrated by " << this->illustrator << ")";
        return ss.str();
    }
}

### Overriding

By declaring a method `string str() const` in the child class, we are **overriding** the method in the base class.

When you call `str()` on a child object, it will call the `str()` method in the child class.

In [4]:
GraphicNovel* graphic = new GraphicNovel("Logicomix: An Epic Search for Truth", "Apostolos Doxiadis", 2009, "Alecos Papadatos");
Book* book = graphic;

In [5]:
graphic->str()

"Logicomix: An Epic Search for Truth by Apostolos Doxiadis (2009) (Illustrated by Alecos Papadatos)"

In [6]:
book->str()

"Logicomix: An Epic Search for Truth by Apostolos Doxiadis (2009)"

When you call an overridden method using a pointer typed to the base class, it calls the base-class method, not the child-class method!

unless...

### `virtual`

In [7]:
class Book {
    private:
    string title;
    string author;
    int year;
    
    public:
    Book(string title, string author, int year) : title(title), author(author), year(year) {}
    Book(Book const& other) : title(other.title), author(other.author), year(other.year) { }
    Book & operator=(Book const& other) {
        this->title = other.title;
        this->author = other.author;
        this->year = other.year;
        return *this;
    }
    
    string getTitle() const { return this->title; }
    string getAuthor() const { return this->author; }
    int getYear() const { return this->year; }
    
    virtual string str() const {  // notice the "virtual" here!
        stringstream ss;
        ss << this->title << " by " << this->author << " (" << this->year << ")";
        return ss.str();
    }
}

In [8]:
// Nothing changed in this class
class GraphicNovel : public Book {
    private:
    string illustrator;
    public:
    GraphicNovel(string title, string author, int year, string illustrator) 
        : Book(title, author, year), illustrator(illustrator) {}
    
    string str() const {
        stringstream ss;
        ss << Book::str() << " (Illustrated by " << this->illustrator << ")";
        return ss.str();
    }
}

In [9]:
GraphicNovel* graphic = new GraphicNovel("Logicomix: An Epic Search for Truth", "Apostolos Doxiadis", 2009, "Alecos Papadatos");
Book* book = graphic;

In [10]:
graphic->str()

"Logicomix: An Epic Search for Truth by Apostolos Doxiadis (2009) (Illustrated by Alecos Papadatos)"

In [11]:
book->str()

"Logicomix: An Epic Search for Truth by Apostolos Doxiadis (2009) (Illustrated by Alecos Papadatos)"

### `virtual` and destructors

If you have a virtual function in your class (i.e. you expect your class to be inherited) then you should declare your destructor to be virtual also.

This way, when an object is deleted using a parent-type pointer, the child-class destructor still gets called.

If only the parent-class destructor were called, then whatever resources the child-class destructor released would remain, resulting in a memory leak.

### Pure Virtual

In [12]:
class Loud {
    public:
    virtual string makeNoise() const = 0;
}

In [13]:
Loud loudThing;

input_line_27:2:7: error: variable type '__cling_N516::Loud' is an abstract class
 Loud loudThing;
      ^
input_line_26:3:20: note: unimplemented pure virtual method 'makeNoise' in 'Loud'
    virtual string makeNoise() const = 0;
                   ^


Interpreter Error: 

Classes with **pure virtual** methods are **abstract**. 

They cannot be instantiated. 

In [14]:
class Dog : public Loud {
    string makeNoise() const { return string("Woof!"); }
}

In [15]:
class Instructor : public Loud {
    string makeNoise() const { return string("Blah blah blah."); }
}

In [16]:
class Student : public Loud {
    string makeNoise() const { return string("Yak yak yak..."); }
}

These classes are not abstract because they implement all the pure virtual methods from the base class.

In [17]:
Loud** loudThings = new Loud*[6];
loudThings[0] = new Dog;
loudThings[1] = new Instructor;
loudThings[2] = new Student;
loudThings[3] = new Instructor;
loudThings[4] = new Student;
loudThings[5] = new Dog;

for (int i = 0; i < 6; i++) {
    cout << loudThings[i]->makeNoise() << endl;
}

Woof!
Blah blah blah.
Yak yak yak...
Blah blah blah.
Yak yak yak...
Woof!


The concept of having many different types used by the same context is called **polymorphism**.

Polymorphism is powerful. 

In [23]:
class Salamander : public Loud {
    void sitAround() {}
}

In [24]:
delete loudThings[5];
loudThings[5] = new Salamander;

for (int i = 0; i < 6; i++) {
    cout << loudThings[i]->makeNoise() << endl;
}

input_line_38:2:2: warning: delete called on '__cling_N516::Loud' that is abstract but has non-virtual destructor [-Wdelete-non-virtual-dtor]
 delete loudThings[5];
 ^
input_line_38:3:21: error: allocating an object of abstract class type '__cling_N527::Salamander'
loudThings[5] = new Salamander;
                    ^
input_line_26:3:20: note: unimplemented pure virtual method 'makeNoise' in 'Salamander'
    virtual string makeNoise() const = 0;
                   ^


Interpreter Error: 

If you don't implement a pure virtual method from the base class, then your class becomes **abstract** and you can't instantiate it.

## A word of advice

C++ has been around for a long time, and so you'll see C++ code that does things in ways that we've since figured out aren't a good idea.

Inheritance is one of those ideas.

Abstract base classes with pure-virtual methods is fine. This defines an **interface** and is a powerful strategy in many languages.

Avoid writing classes where the **functionality** of the class requires looking at more than one class definition. Certainly, avoid overriding non-virtual methods.

If you want to modularize your code, use **composition** instead of inheritance.

[Youtube: Sandi Metz "Nothing is Something"](https://www.youtube.com/watch?v=OMPfEXIlTVE)

## Key Ideas

- Inheritance
- `virtual`
- Polymorphism
- Abstract Base Classes (ABCs)